In [1]:
import statsmodels.formula.api as smf
import fastreg as fr
from fastreg import I, R, C, C0

### Generate Data

In [2]:
models = ['linear', 'poisson', 'logit', 'negbin']
data = fr.dataset(N=1_000_000, K1=10, K2=100, models=models, seed=89320432)
data_wide = fr.dataset(N=1_000_000, K1=10, K2=10_000, models=models, seed=89320433)
data.head()

,x1,x2,yhat0,yhat,id1,id2,y0,y,Eb0,Eb,b0,b,Ep0,Ep,p0,p,nb0,nb
0,-1.429225,1.830295,0.769410,2.149410,H,68,1.791671,1.585510,0.683393,0.895614,1,1,2.158492,8.579792,4,10,1,5
1,0.687153,0.795373,0.783369,1.403369,C,42,0.295552,3.163529,0.686406,0.802718,1,1,2.188835,4.068886,1,2,1,0
2,0.764353,-1.060225,-0.306829,-0.036829,C,7,0.381637,-0.639362,0.423889,0.490794,1,1,0.735777,0.963841,0,1,2,0
3,-0.309887,-0.777701,-0.459587,0.800413,D,96,0.143218,1.139854,0.387084,0.690063,0,1,0.631545,2.226461,0,0,1,3
4,-0.616042,-0.752266,-0.536172,0.623828,H,46,-0.468395,1.462211,0.369078,0.651089,1,1,0.584983,1.866057,0,1,1,7


In [3]:
# for statsmodels runs
data1 = data.copy()
data1['id2'] = data1['id2'].astype(str)

### Normal OLS

In [4]:
%time fr.ols(y=R.y0, x=I+R.x1+R.x2, data=data)

CPU times: user 44.8 ms, sys: 90.6 ms, total: 135 ms
Wall time: 22.8 ms


y0,coeff,stderr,low95,high95,pvalue
I,0.101859,0.001000,0.099900,0.103818,0.0
x1,0.301647,0.001000,0.299688,0.303606,0.0
x2,0.599408,0.000999,0.597450,0.601366,0.0


In [5]:
%time smf.ols('y0 ~ 1 + x1 + x2', data=data).fit().params

CPU times: user 340 ms, sys: 1.1 s, total: 1.44 s
Wall time: 130 ms


Intercept    0.101859
x1           0.301647
x2           0.599408
dtype: float64

In [6]:
%time fr.ols(y=R.y, x=I+R.x1+R.x2+C.id1+C.id2, data=data)

CPU times: user 430 ms, sys: 772 ms, total: 1.2 s
Wall time: 311 ms


y,coeff,stderr,low95,high95,pvalue
I,0.112463,0.010413,0.092054,0.132872,0.0
x1,0.299557,0.001001,0.297595,0.301518,0.0
x2,0.601840,0.001000,0.599880,0.603800,0.0
id1=B,0.104734,0.004476,0.095962,0.113507,0.0
id1=C,0.201965,0.004474,0.193197,0.210734,0.0
...,...,...,...,...,...
id2=95,0.937461,0.014102,0.909821,0.965101,0.0
id2=96,0.956624,0.014109,0.928970,0.984278,0.0
id2=97,0.960951,0.014099,0.933318,0.988584,0.0
id2=98,0.993398,0.014132,0.965699,1.021097,0.0


In [7]:
%time smf.ols('y ~ 1 + x1 + x2 + id1 + id2', data=data1).fit().params

CPU times: user 1min 38s, sys: 15 s, total: 1min 53s
Wall time: 10.7 s


Intercept    0.112463
id1[T.B]     0.104734
id1[T.C]     0.201965
id1[T.D]     0.300212
id1[T.E]     0.403094
               ...   
id2[T.97]    0.960951
id2[T.98]    0.993398
id2[T.99]    0.988670
x1           0.299557
x2           0.601840
Length: 111, dtype: float64

### High Dimensional

In [8]:
%time fr.ols(y=R.y, x=I+R.x1+R.x2+C.id1+C.id2, data=data_wide)

CPU times: user 4.1 s, sys: 2.5 s, total: 6.6 s
Wall time: 4.51 s


y,coeff,stderr,low95,high95,pvalue
I,0.143877,0.103699,-0.059369,0.347123,1.653041e-01
x1,0.301630,0.001004,0.299662,0.303597,0.000000e+00
x2,0.599284,0.001006,0.597313,0.601255,0.000000e+00
id1=B,0.097642,0.004499,0.088824,0.106461,0.000000e+00
id1=C,0.189008,0.004491,0.180207,0.197810,0.000000e+00
...,...,...,...,...,...
id2=9995,1.045182,0.146196,0.758643,1.331722,8.730794e-13
id2=9996,1.029893,0.146587,0.742588,1.317198,2.128075e-12
id2=9997,0.892911,0.142021,0.614554,1.171267,3.233234e-10
id2=9998,0.961372,0.144705,0.677755,1.244989,3.059886e-11


In [9]:
%time fr.ols(y=R.y, x=I+R.x1+R.x2+C.id1, hdfe=C.id2, data=data_wide)

CPU times: user 354 ms, sys: 49.6 ms, total: 404 ms
Wall time: 354 ms


y,coeff,stderr,low95,high95,pvalue
I,0.143877,0.103699,-0.059369,0.347123,1.653041e-01
x1,0.301630,0.001004,0.299662,0.303597,0.000000e+00
x2,0.599284,0.001006,0.597313,0.601255,0.000000e+00
id1=B,0.097642,0.004499,0.088824,0.106461,0.000000e+00
id1=C,0.189008,0.004491,0.180207,0.197810,0.000000e+00
...,...,...,...,...,...
id2=9995,1.045182,0.146196,0.758643,1.331722,8.730794e-13
id2=9996,1.029893,0.146587,0.742588,1.317198,2.128075e-12
id2=9997,0.892911,0.142021,0.614554,1.171267,3.233234e-10
id2=9998,0.961372,0.144705,0.677755,1.244989,3.059886e-11


In [10]:
%time fr.ols(y=R.y, x=I+R.x1+R.x2+C.id1, absorb=C.id2, data=data_wide)

CPU times: user 793 ms, sys: 1.87 s, total: 2.67 s
Wall time: 363 ms


y,coeff,stderr,low95,high95,pvalue
I,0.605524,0.003035,0.599576,0.611471,0.0
x1,0.301630,0.000998,0.299674,0.303585,0.0
x2,0.599284,0.001014,0.597297,0.601270,0.0
id1=B,0.097642,0.004531,0.088762,0.106523,0.0
id1=C,0.189008,0.004545,0.180100,0.197917,0.0
id1=D,0.295160,0.004506,0.286328,0.303991,0.0
id1=E,0.402802,0.004510,0.393963,0.411641,0.0
id1=F,0.495409,0.004493,0.486602,0.504215,0.0
id1=G,0.596910,0.004467,0.588155,0.605665,0.0
id1=H,0.694326,0.004490,0.685526,0.703125,0.0


### Poisson

In [11]:
%time fr.poisson(y=R.p0, x=I+R.x1+R.x2, data=data)

[  0] ℓ=-5348.71533, g=1584.57983, Δβ=0.58524, Δℓ=inf, μR=0.33294, μC=nan
[ 10] ℓ=-5097.07080, g=0.19439, Δβ=0.00000, Δℓ=0.00000, μR=0.33668, μC=nan
CPU times: user 1.59 s, sys: 249 ms, total: 1.84 s
Wall time: 1.57 s


p0,coeff,stderr,low95,high95,pvalue
I,0.105487,0.001023,0.103483,0.107491,0.0
x1,0.306340,0.000849,0.304677,0.308004,0.0
x2,0.598214,0.000847,0.596554,0.599875,0.0


In [12]:
%time fr.poisson(y=R.p, x=I+R.x1+R.x2+C.id1+C.id2, data=data)

[  0] ℓ=18348.32227, g=4987.58691, Δβ=0.71672, Δℓ=inf, μR=0.44834, μC=0.32629
[ 20] ℓ=20926.96094, g=2.20907, Δβ=0.00513, Δℓ=0.09180, μR=0.36513, μC=0.45355
[ 40] ℓ=20927.66406, g=0.56862, Δβ=0.00163, Δℓ=0.00781, μR=0.34542, μC=0.48320
[ 60] ℓ=20927.74609, g=0.14162, Δβ=0.00054, Δℓ=0.00000, μR=0.33902, μC=0.49283
[ 80] ℓ=20927.75586, g=0.14200, Δβ=0.00018, Δℓ=0.00195, μR=0.33688, μC=0.49604
[ 91] ℓ=20927.75391, g=0.15004, Δβ=0.00010, Δℓ=0.00000, μR=0.33640, μC=0.49677
CPU times: user 16.4 s, sys: 2.82 s, total: 19.2 s
Wall time: 12.3 s


p,coeff,stderr,low95,high95,pvalue
I,0.106851,0.006941,0.093246,0.120456,0.0
x1,0.297527,0.000508,0.296532,0.298523,0.0
x2,0.604811,0.000506,0.603819,0.605802,0.0
id1=B,0.097337,0.002832,0.091787,0.102887,0.0
id1=C,0.200588,0.002770,0.195158,0.206017,0.0
...,...,...,...,...,...
id2=95,0.945656,0.007787,0.930394,0.960918,0.0
id2=96,0.951543,0.007816,0.936225,0.966862,0.0
id2=97,0.955921,0.007822,0.940591,0.971252,0.0
id2=98,0.988599,0.007781,0.973349,1.003849,0.0


### Logit

In [13]:
%time fr.logit(y=R.b0, x=I+R.x1+R.x2, data=data)

[  0] ℓ=-5306.84863, g=247.49107, Δβ=0.58688, Δℓ=inf, μR=0.33164, μC=nan
[  6] ℓ=-5270.57373, g=0.05111, Δβ=0.00009, Δℓ=0.00000, μR=0.33756, μC=nan
CPU times: user 922 ms, sys: 126 ms, total: 1.05 s
Wall time: 723 ms


b0,coeff,stderr,low95,high95,pvalue
I,0.098849,0.002104,0.094725,0.102973,0.0
x1,0.317276,0.002146,0.313070,0.321481,0.0
x2,0.596545,0.002269,0.592097,0.600992,0.0


In [14]:
%time fr.logit(y=R.b, x=I+R.x1+R.x2+C.id1+C.id2, data=data)

[  0] ℓ=-4558.50586, g=200.64862, Δβ=0.66172, Δℓ=inf, μR=0.42488, μC=0.32683
[ 20] ℓ=-4455.45068, g=0.14746, Δβ=0.00268, Δℓ=0.00342, μR=0.38001, μC=0.42297
[ 40] ℓ=-4455.39404, g=0.07907, Δβ=0.00166, Δℓ=0.00146, μR=0.36603, μC=0.44419
[ 60] ℓ=-4455.37549, g=0.05408, Δβ=0.00102, Δℓ=0.00000, μR=0.35740, μC=0.45729
[ 80] ℓ=-4455.37109, g=0.05401, Δβ=0.00063, Δℓ=0.00049, μR=0.35208, μC=0.46536
[ 99] ℓ=-4455.36768, g=0.05398, Δβ=0.00040, Δℓ=0.00098, μR=0.34894, μC=0.47012
CPU times: user 17.7 s, sys: 2.25 s, total: 19.9 s
Wall time: 13 s


b,coeff,stderr,low95,high95,pvalue
I,0.142164,0.022841,0.097397,0.186932,4.843999e-10
x1,0.303466,0.002382,0.298798,0.308134,0.000000e+00
x2,0.601201,0.002508,0.596285,0.606116,0.000000e+00
id1=B,0.076194,0.009834,0.056919,0.095469,9.325873e-15
id1=C,0.197577,0.009910,0.178154,0.217000,0.000000e+00
...,...,...,...,...,...
id2=95,0.818947,0.033489,0.753310,0.884584,0.000000e+00
id2=96,0.919102,0.034055,0.852356,0.985848,0.000000e+00
id2=97,0.915509,0.033866,0.849131,0.981886,0.000000e+00
id2=98,0.917561,0.034024,0.850876,0.984246,0.000000e+00


### Ultra Wide

In [15]:
N = 5_000_000
df = pd.DataFrame({ 
    'x1': np.random.rand(N), 
    'x2': np.random.rand(N), 
    'id1': np.ceil(10*np.arange(N)/N+1e-7).astype(int),
    'id2': np.random.randint(1, 100_001, size=N)
})
df['y'] = 1 + 2*df['x1'] + 3*df['x2'] + np.log10(df['id1']) + np.log10(df['id2']) + np.random.randn(N)
print(df[['id1', 'id2']].nunique())

id1        10
id2    100000
dtype: int64


In [16]:
%time fr.ols(y=R.y, x=I+R.x1+R.x2+C.id1, hdfe=C.id2, data=df)

CPU times: user 2 s, sys: 841 ms, total: 2.84 s
Wall time: 1.97 s


y,coeff,stderr,low95,high95,pvalue
I,0.917833,0.136084,0.651113,1.184552,1.534439e-11
x1,2.000167,0.001565,1.997100,2.003235,0.000000e+00
x2,2.999236,0.001565,2.996169,3.002303,0.000000e+00
id1=2,0.298825,0.002020,0.294866,0.302785,0.000000e+00
id1=3,0.475838,0.002020,0.471878,0.479798,0.000000e+00
...,...,...,...,...,...
id2=99996,5.024529,0.196249,4.639889,5.409170,0.000000e+00
id2=99997,4.946542,0.190712,4.572754,5.320330,0.000000e+00
id2=99998,5.290023,0.190712,4.916234,5.663811,0.000000e+00
id2=99999,4.935762,0.197286,4.549089,5.322435,0.000000e+00
